In [1]:
import pymongo
import re
import numpy
import pandas as pd
import json
from urllib.request import urlopen, quote

client = pymongo.MongoClient(host='localhost', port=27017)
sina = client['Sina']
comments_information = sina['Comments_Information']

# 位置分布

In [2]:
location_count = {}
for data in comments_information.find():
    if 'city' in data.keys():
        location = data['province']+data['city']
        if location in location_count:
            location_count[location] += 1
        else:
            location_count[location] = 1
    elif 'province' in data.keys():
        location = data['province']
        if location in location_count:
            location_count[location] += 1
        else:
            location_count[location] = 1

In [3]:
location_count_sort = sorted(location_count.items(), key=lambda item: int(item[1]), reverse=True)
print(location_count_sort)

[('其他', 67062), ('海外', 7799), ('四川成都', 7289), ('北京', 6445), ('江苏', 4393), ('四川', 4300), ('广东', 4047), ('湖北武汉', 3791), ('上海', 3578), ('山东', 3485), ('广东广州', 3360), ('重庆', 3217), ('江苏南京', 3217), ('浙江', 3048), ('浙江杭州', 2754), ('广东深圳', 2638), ('陕西西安', 2627), ('湖北', 2614), ('湖南长沙', 2241), ('湖南', 2075), ('河南', 2040), ('河南郑州', 1884), ('海外美国', 1828), ('江苏苏州', 1771), ('安徽', 1700), ('山东青岛', 1525), ('福建', 1509), ('海外其他', 1508), ('陕西', 1442), ('山东济南', 1392), ('天津', 1363), ('北京海淀区', 1281), ('北京东城区', 1239), ('安徽合肥', 1234), ('福建福州', 1232), ('河北', 1221), ('海外日本', 1195), ('江西', 1172), ('辽宁', 1158), ('浙江宁波', 1113), ('北京朝阳区', 1051), ('辽宁沈阳', 1036), ('江西南昌', 1001), ('黑龙江哈尔滨', 983), ('江苏无锡', 974), ('浙江温州', 973), ('福建厦门', 956), ('广西南宁', 904), ('广西', 886), ('辽宁大连', 849), ('山西', 789), ('上海浦东新区', 774), ('黑龙江', 741), ('云南昆明', 739), ('河北石家庄', 718), ('吉林长春', 697), ('福建泉州', 694), ('山西太原', 694), ('香港', 668), ('四川绵阳', 662), ('云南', 657), ('海外英国', 625), ('江苏徐州', 624), ('广东东莞', 620), ('海外澳大利亚', 615), ('江苏常州', 614), ('广东

In [4]:
def query_coord(location):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    ak = 'xxx'
    add = quote(location)
    output = 'json'
    query_url = url + '?' + 'address=' + add + '&output=' + output + '&ak=' + ak
    request = urlopen(query_url)
    response = request.read().decode()
    result = json.loads(response)
    if result['status']!=0:
        return None, None
    else:
        level = result['result']['level'] 
        # 提取经度
        lng = result['result']['location']['lng']
        # 提取纬度
        lat = result['result']['location']['lat']
        # 写成新的字典
        return {"lng": lng, "lat": lat}, level

In [5]:
query_coord('台南')

({'lng': 121.36052796411255, 'lat': 38.96584875626212}, 'UNKNOWN')

In [ ]:
level_filter = ['国家', '省', '城市', '区县']
heatmap_data = []
for location_count in location_count_sort:
    location,count = location_count
    coord, level = query_coord(location)
    if level in level_filter and coord is not None:
        coord['count'] = count
        heatmap_data.append(coord)

In [ ]:
heatmap_data

In [7]:
def data_to_csv(data, name, label, normalize = True):
    data = [list(d) for d in data]
    if normalize:
        data_sum = sum([d[1] for d in data])
        for d in data:
            d[1] /= data_sum
    if label:
        data.insert(0, label)
    data = pd.DataFrame(data)
    data.to_csv(name + '.csv',header = False, index = False, encoding = 'utf-8-sig')
    return data

In [9]:
data_to_csv(location_count_sort, '地理位置', ['地点','人数'], normalize = False)

,0,1
0,地点,人数
1,其他,67062
2,海外,7799
3,四川成都,7289
4,北京,6445
5,江苏,4393
6,四川,4300
7,广东,4047
8,湖北武汉,3791
9,上海,3578
